# OpenAI connection (not free)

In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
client=OpenAI()

In [ ]:
client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content":"is it too late to joint the course ?"}]
)

# GroQ connection (free)

In [13]:
import yaml

# Open the file
with open('api_keys.yml', 'r') as file:
    # Load the data from the file
    data = yaml.safe_load(file)
    
# Get the API key (Groq example here)
groq_api_key = data['GROQ_API_KEY']

In [14]:
# import libraries
from groq import Groq
import os

# create client calling Groq class
client = Groq(api_key=groq_api_key)

# create a query
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Is it too late to join the course?",
        }
    ],
    model="llama3-8b-8192",
)

# print the response
print(response.choices[0].message.content)

I'm happy to help you with your question! However, I need more information to provide a accurate answer. Can you please tell me which course you are referring to and is there any specific deadline or window to enroll in that course?


# Homework week 1

In [1]:
from elasticsearch import Elasticsearch
import requests 
from tqdm import tqdm
import tiktoken

In [2]:
# Import needed documents (FAQ DE zoomcamp)

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

Before running Elastisearch, we must run the associated docker:
```
docker run -it     --rm     --name elasticsearch     -p 9200:9200     -p 9300:9300     -e "discovery.type=single-node"     -e "xpack.security.enabled=false"     docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [4]:
# Create ElasticSearch client after connection in VSCode terminal
es_client = Elasticsearch("http://localhost:9200")

In [5]:
#Create an index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [6]:
index_name = "course_questions"

In [7]:
# If we need to delete all indices:
!curl -X DELETE "http://localhost:9200/course_questions"


{"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [course_questions]","resource.type":"index_or_alias","resource.id":"course_questions","index_uuid":"_na_","index":"course_questions"}],"type":"index_not_found_exception","reason":"no such index [course_questions]","resource.type":"index_or_alias","resource.id":"course_questions","index_uuid":"_na_","index":"course_questions"},"status":404}

In [8]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:19<00:00, 49.54it/s]


In [18]:
def elastic_search(query, course=None):

    if course:
        print(f"Filtering answers on course: {course}") 
        search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": course
                        }
                    }
                }
            }
        }
    else:
        search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"],
                            "type": "best_fields"
                        }
                    },
                }
            }
        }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    result_scores = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        result_scores.append(hit['_score'])

    return response, result_docs, result_scores

## Question 3

In [11]:
query = "How do I execute a command in a running docker container?"

In [12]:
response_query, top_doc, top_score = elastic_search(query)

In [13]:
response_query

ObjectApiResponse({'took': 48, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course_questions', '_id': 'RwuXQJABYP5LKfuGm5T5', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course_questions', '_id': 'VQuXQJABYP5LKfuGdJKo', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker 

In [15]:
top_score[0]

84.050095

## Question 4

In [49]:
query = "How do I execute a command in a running docker container?"

In [50]:
response_query, top_doc, top_score = elastic_search(query, course = "machine-learning-zoomcamp")

Filtering answers on course: machine-learning-zoomcamp


In [51]:
top_doc

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [52]:
top_doc[:3]

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [53]:
# Get the third hit question
response_query['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

## Question 5

In [77]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [86]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context_entry = context_template.format(question=doc['question'], text=doc['text'])
        context = context + context_entry + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [87]:
query = "How do I execute a command in a running docker container?"
search_results = top_doc[:3]

In [88]:
search_results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [89]:
prompt_template = build_prompt(query, search_results)

In [90]:
prompt_template

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n    \n    QUESTION: How do I execute a command in a running docker container?\n    \n    CONTEXT:\n    Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /p

In [91]:
len(prompt_template)

1486

## Question 6

In [92]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [93]:
len(encoding.encode(prompt_template))

328